In [23]:
from pptx import Presentation
import xlsxwriter, time
path = "D:\\Code\\Project\\CrawlerProj_Anson_Python\\ettoday_SVN\\RD Document\\"
fileName = '0.醫材研討會議程.pptx'

In [24]:
''' 讀pptx檔轉txt'''    
def GetallText(fileName):
    prs = Presentation(fileName)
    AllTextList = []
    for slide in prs.slides:
        for shape in slide.shapes:
            if not shape.has_text_frame: #判斷shape是否為空 
                continue
            txt = shape.text # 這裡一定要用變數存shape.text，不然會錯!
            try:
                #print(shape.text)
                if txt !=' 'and txt !='': 
                    AllTextList.append(txt)
            except AttributeError:
                print("error")
                continue
    return AllTextList

In [25]:
AllText_list = GetallText(path+fileName) 
# AllText_list = ['Classify:AUO-Confidential\', 'RRCAA3 Monthly Report – Num 2018', B_FSI = 1.5~3 ; BW = 1',...]

AllText_str = ''.join(AllText_list)
AllText_str

'醫材技術 研討會\u200b\u200cClassify:AUO-General\u200c\u200b醫材技術 研討會時間: 2018年12月12日\n地點: Qisda 桃園總部 A70\n主辦單位: 明基友達集團 GRD 醫材委員會\u200b\u200cClassify:AUO-General\u200c\u200b\u200b\u200cClassify:AUO-General\u200c\u200b\u200b\u200cClassify:AUO-General\u200c\u200b'

In [29]:
dict = {'Name': 'Zara', 'Age': 27}
print ("Value : %s" %dict.get('Age'))

Value : 27


In [26]:
import logging
import jieba
from operator import itemgetter

logging.basicConfig(level = logging.DEBUG, format=' %(asctime)s - %(levelname)s - %(message)s')
logging.debug('Start of program')
sentence = AllText_str 
topK=10
withWeight=True
allowPOS=()
withFlag=False
tokenizer = jieba.dt
stop_words = set((
        "the", "of", "is", "and", "to", "in", "that", "we", "for", "an", "are",
        "by", "be", "as", "on", "with", "can", "if", "from", "which", "you", "it",
        "this", "then", "at", "have", "all", "not", "one", "has", "or", "that"
    ))
idf_freq = {}
median_idf = 0.0

if allowPOS:
    allowPOS = frozenset(allowPOS)
    words = postokenizer.cut(sentence)
else:
    words = tokenizer.cut(sentence)
freq = {}
freq_cnt = {}
# logging.DEBUG('顯示freq  = %s' %freq)
for w in words:
    if allowPOS:
        if w.flag not in allowPOS:
            continue
        elif not withFlag:
            w = w.word
    wc = w.word if allowPOS and withFlag else w
    #if len(wc.strip()) < 2 or wc.lower() in self.stop_words or wc.lower()[0].encode('UTF-8').isalpha() or wc.lower()[0].isdigit():  # HL 拿掉數字和英文
    #if len(wc.strip()) < 2 or wc.lower() in self.stop_words or wc.lower()[0].isdigit():  # Anson 保留英文/拿掉數字
    if len(wc.strip()) < 2 or wc.lower() in stop_words:  # Anson 保留英文/數字/中文
        continue
    freq[w] = freq.get(w, 0.0) + 1.0
    freq_cnt[w] = freq_cnt.get(w, 0.0) + 1.0
    #logging.DEBUG(level=logging.DEBUG, format='顯示freq %s' %(freq))
total = sum(freq.values())
print('顯示total字個數 = %d\n' %total)
for k in freq:
    kw = k.word if allowPOS and withFlag else k
    freq[k] *= idf_freq.get(kw, median_idf) / total

if withWeight:
    tags = sorted(freq.items(), key=itemgetter(1), reverse=True)
    freq_cnt_tags = freq_cnt.items()
else:
    tags = sorted(freq, key=freq.__getitem__, reverse=True)
if topK:
    print('tags[:topK] = %s, \nfreq_cnt_tags = %s' %(tags[:topK], freq_cnt_tags))
    #print('freq_cnt_tags = %s' %(freq_cnt_tags[tags[:topK]]))
# else:
#     #print(tags)

 2019-01-04 13:10:27,833 - DEBUG - Start of program


顯示total字個數 = 35

tags[:topK] = [('友達', 0.0), ('集團', 0.0), ('總部', 0.0), ('時間', 0.0), ('委員會', 0.0), ('AUO', 0.0), ('Classify', 0.0), ('研討會', 0.0), ('技術', 0.0), ('桃園', 0.0)], 
freq_cnt_tags = dict_items([('友達', 1.0), ('集團', 1.0), ('總部', 1.0), ('時間', 1.0), ('委員會', 1.0), ('AUO', 4.0), ('Classify', 4.0), ('研討會', 2.0), ('技術', 2.0), ('桃園', 1.0), ('GRD', 1.0), ('單位', 1.0), ('主辦', 1.0), ('A70', 1.0), ('2018', 1.0), ('12', 2.0), ('Qisda', 1.0), ('General', 4.0), ('醫材', 3.0), ('地點', 1.0), ('明基', 1.0)])


In [27]:
tags[:topK]

[('友達', 0.0),
 ('集團', 0.0),
 ('總部', 0.0),
 ('時間', 0.0),
 ('委員會', 0.0),
 ('AUO', 0.0),
 ('Classify', 0.0),
 ('研討會', 0.0),
 ('技術', 0.0),
 ('桃園', 0.0)]

In [5]:
import logging
logging.basicConfig(level = logging.DEBUG, format=' %(asctime)s - %(levelname)s - %(message)s')

logging.debug('Start of program')
def extract_tags(sentence, topK=20, withWeight=False, allowPOS=(), withFlag=False):
        """
        Extract keywords from sentence using TF-IDF algorithm.
        Parameter:
            - topK: return how many top keywords. `None` for all possible words.
            - withWeight: if True, return a list of (word, weight);
                          if False, return a list of words.
            - allowPOS: the allowed POS list eg. ['ns', 'n', 'vn', 'v','nr'].
                        if the POS of w is not in this list,it will be filtered.
            - withFlag: only work with allowPOS is not empty.
                        if True, return a list of pair(word, weight) like posseg.cut
                        if False, return a list of words
        """
        if allowPOS:
            allowPOS = frozenset(allowPOS)
            words = self.postokenizer.cut(sentence)
        else:
            words = self.tokenizer.cut(sentence)
        freq = {}
        logging.basicConfig('顯示freq  = %s' %(freq))
        for w in words:
            if allowPOS:
                if w.flag not in allowPOS:
                    continue
                elif not withFlag:
                    w = w.word
            wc = w.word if allowPOS and withFlag else w
            #if len(wc.strip()) < 2 or wc.lower() in self.stop_words or wc.lower()[0].encode('UTF-8').isalpha() or wc.lower()[0].isdigit():  # HL 拿掉數字和英文
            #if len(wc.strip()) < 2 or wc.lower() in self.stop_words or wc.lower()[0].isdigit():  # Anson 保留英文/拿掉數字
            if len(wc.strip()) < 2 or wc.lower() in self.stop_words:  # Anson 保留英文/數字/中文
                continue
            freq[w] = freq.get(w, 0.0) + 1.0
            logging.basicConfig(level=logging.DEBUG, format='顯示freq %s' %(freq))
        total = sum(freq.values())
        print('total = %s' %total)
        for k in freq:
            kw = k.word if allowPOS and withFlag else k
            freq[k] *= self.idf_freq.get(kw, self.median_idf) / total

        if withWeight:
            tags = sorted(freq.items(), key=itemgetter(1), reverse=True)
        else:
            tags = sorted(freq, key=freq.__getitem__, reverse=True)
        if topK:
            return tags[:topK]
        else:
            return tags

 2019-01-02 10:13:31,778 - DEBUG - Start of program


In [6]:
import jieba.analyse, logging
sentence = AllText_str
tfidf_list = jieba.analyse.extract_tags(sentence, topK=50, withWeight = True, allowPOS=())
# tfidf_list = extract_tags(sentence, topK=50, withWeight = True, allowPOS=())


 2019-01-02 10:13:49,298 - DEBUG - Start of TF-IDF program


有用jieba有统计好的idf值
顯示total斷字個數 = 780

顯示IDF_all斷字個數 = {'量測': 11.3357282945, '可能': 11.3357282945, 'Panel': 11.3357282945, '阻值': 11.3357282945, 'MainSubMainw': 11.3357282945, 'TD': 11.3357282945, '45': 11.3357282945, 'BM': 11.3357282945, 'Washout': 11.3357282945, 'Improve': 11.3357282945, '17DVT1': 11.3357282945, '不同': 11.3357282945, 'Fast': 11.3357282945, 'Open': 11.3357282945, '問題': 3.96351357308, '評估': 6.91688768672, '開關': 8.40350942668, 'umLC': 11.3357282945, '形變': 9.91169360541, '因為': 4.04837813621, 'FRT': 11.3357282945, 'RRCAA3': 11.3357282945, '損失': 11.3357282945, '造成': 11.3357282945, '需先給': 11.3357282945, '前提': 6.58081272217, '5000A': 11.3357282945, '強烈': 11.3357282945, 'gapNon': 11.3357282945, 'O10': 11.3357282945, 'UV1': 11.3357282945, 'com': 7.89989532, 'DPx': 11.3357282945, '側視': 11.0103058941, 'washout': 11.3357282945, 'Taiwan': 11.3357282945, '1716LC': 11.3357282945, 'RT': 11.3357282945, 'Classify': 11.3357282945, 'SB': 11.3357282945, 'Cell': 11.3357282945, '改善': 5.7296434627

([('Classify', 0.31972566984487183),
  ('Confidential', 0.31972566984487183),
  ('AUO', 0.31972566984487183),
  ('NFA', 0.30519268485192314),
  ('PSA', 0.24706074488012822),
  ('COM', 0.20346178990128205),
  ('ITO', 0.15986283492243591),
  ('Electrode', 0.1453298499294872),
  ('High', 0.1453298499294872),
  ('Design', 0.1453298499294872),
  ('Cell', 0.13079686493653847),
  ('curing', 0.11626387994358975),
  ('液晶', 0.10173089495064103),
  ('PX', 0.10173089495064103),
  ('COA', 0.10173089495064103),
  ('地形', 0.10173089495064103),
  ('New', 0.10173089495064103),
  ('設計', 0.08719790995769232),
  ('LCA', 0.08719790995769232),
  ('Slit', 0.08719790995769232),
  ('AR', 0.08719790995769232),
  ('Tr', 0.08719790995769232),
  ('FRT', 0.0726649249647436),
  ('20V', 0.0726649249647436),
  ('Domain', 0.0726649249647436),
  ('電壓', 0.0726649249647436),
  ('com', 0.07089649646153845),
  ('BM', 0.058131939971794874),
  ('10', 0.058131939971794874),
  ('CR', 0.058131939971794874),
  ('Test', 0.058131939